# Regresing out the Voxel wise EVs (from Pnm) and motion parameters
You need to creat blanck design file for each subjects. It must done manually. Yo this you can use Glm.
run Glm. in GLM setup windows: adjust number of time points(990), TR (4s), and high path temporal filtering (100s). Then in General Linear Model window: select Basic shap as Empty(all zeros). At the end save them as "blank_desing" in design folder inside results directory for each subject.

-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import os
import glob

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
sub_dirs = glob.glob('%s/sub*/2*/results'%(data_path))

for curr_sub in sub_dirs:
    
    print('Current subject is: %s'%(curr_sub.split('/')[-3:-1]))
    
    os.chdir(curr_sub)
    
    func_file = 'preprocessed_data_highpassed_chopped.nii.gz'# or preprocessed_data_bandpassed_chopped.nii
    mask_file = os.path.join('reg','example_func_mask.nii.gz')
    
    print('\t 1) calculating the minimum intensity of the function data ...')
    thr = os.popen("fslstats %s -k %s -R | awk '{ print  }' -"%(func_file,mask_file)).read().rstrip()
    thr = float(thr.split(' ')[0])
    print('\t\t The minimum intensity is %s.'%(thr))
    
    out_put = 'CSF_analysis_highPass'
    
    print('\t 2) Regressing out the voxelwise EVs of respration signal ...')
    os.system("film_gls --in={1} --rn={2} --pd=design/blank_design.mat --thr={0} --sa --ms=5 --con=design/blank_design.con --ven=1,2,3,4,5,6,7,8,9 --vef=PNM/mypnmev001,PNM/mypnmev002,PNM/mypnmev003,PNM/mypnmev004,PNM/mypnmev005,PNM/mypnmev006,PNM/mypnmev007,PNM/mypnmev008,PNM/mypnmev009".format(str(thr),func_file,out_put))
    
    print('\t 3) Calculating mean_fucn and add it to the residual ...')
    os.system('fslmaths %s -Tmean %s/mean_func'%(func_file,out_put))    
    os.system("fslmaths {0}/res4d -add {0}/mean_func {0}/res4d_mean".format(out_put))
    
    print('\t 4) Regressing out the motion parameters ...')
    os.system('fsl_regfilt -i {0}/res4d_mean -o {0}/denoised_data -d mc/bold_mcf_chopped.par -f "1,2,3,4,5,6"'.format(out_put))